In [26]:
# main.ipynb
# Archivo base en el que se maneja la experimentación del diseño metodológico.
# Objetivo general: Utilizar los datos publicos (ubicados en la carpeta /data/)
#     de municipios en los que existe información para diseñar una estrategia
#     para distribuir alimentos en el territorio nacional.
# Objetivos específicos:

#     ---pronostico_poblacional---
#     [IA]1. Determinar por municipio que herramienta es mejor para pronosticar
#            la población en 10 años a partir del año actual.
#               a. Regresión Lineal Multiple || Multiple Linear Regression.
#               b. Arboles de Regresión || Regression Tree.
#               c. Máquinas de Vectores de Soporte || Support Vector Machine.
#               d. Bosques Aleatorios || Random Forest Regression.
#               e. Redes neurales || Deep Learning.
#               f. Regresión Tradicional.
#     ---capacidad_y_costo---
#     [IO]2. Determinar las capacidades y costos de almacenamiento de alimentos
#            en cada municipio utilizando los datos del pronóstico, los datos
#            abiertos y consideraciones de los analístas.
#     ---pronosticar_capacidad_y_costo---
#     [IA]3. Entrenar un clasificador desde los datos de capacidades y costos
#            de almacenamiento por municipio.
#               a. Arboles de Decision || Decision Tree.
#               b. Análisis Discriminante Lineal || Linear Discriminant Analysis.
#               c. Regresión Logística || Logistic Regression.
#               d. Máquinas de Vectores de Soporte || Support Vector Machine.
#               e. Redes Neurales || Deep Learning.
#               f. Análisis de frecuencias. [Aún está en desarrollo la idea]
#     ---cantidad_de_clusteres---
#     [IO]4. Proponer una cantidad de clusteres que permitan disminuir el costo
#            computacional del algoritmo optimizador y compararlo con los métodos
#            tradicionales.
#     ---generar_clusteres---
#     [IA]5. Dividir los municipios en clústeres, rectificando que es viable
#            satisfacer la demanda de alimentos con la capacidad instalada del
#            municipio.
#               a. k-means.
#               b. Mapa Autoorganizado || Self-Organizing Map.
#               c. Agrupamiento Jerárquico || Agglomerative Clustering.
#               d. DBSCAN.
#     ---solucionar_cflp---
#     [IO]6. Resolver el cflp para diferentes escenarios.
#               a. Solución ingenua (todos los municipios tienen un centro de
#                  distribución).
#               b. Datos completos sin clusterizar.
#               c. Dividido por clústers.
#


In [27]:
# Manejo básico de archivos
import os, sys, warnings

# Registro del tiempo
import time

# Manejo de datos
import pandas as pd
import numpy as np

# Funciones personalizadas de /funciones/funciones.py, /funciones/alistamiento.py, /funciones/miscelaneas.py
from funciones.funciones import *
from funciones.alistamiento import *
from funciones.miscelaneas import *

# funciones del flujo de trabajo
from funciones.pronostico_poblacional import *
from funciones.capacidad_y_costo import *

import pulp as pl


In [28]:
# Funciones


In [29]:
# PARAMETROS


from math import log


comida_per_capita = 0.00087617  # 0.87617 kg por persona / día
densidad_de_alimentos = 537 / 1000  # 537 kg por m3 (5 metros de altura)
RANDOM_SEED = 11

# crear_estructura_de_archivos()
# procesar_datos_completos(comida_per_capita, densidad_de_alimentos)
# procesar_datos_imperfectos()
# pronostico_poblacional(RANDOM_SEED)

# capacidad_y_costo()
# 2. Determinar las capacidades y costos de almacenamiento de alimentos
# en cada municipio utilizando los datos del pronóstico, los datos
# abiertos y consideraciones de los analístas.

# en esta sección, se busca encontrar la capacidad y el costo de almacenamiento
# de alimentos para cada municipio, ya se tiene en los datos de almacenes la
# capacidad y costo mensual de algunos municipios, para los cuales no se puede
# alquilar un almacén, se asume que la mejor estratégia posible es construir
# naves industriales, los tipos de almacenes son (todos de 5m de altura):
#     tipo | capacidad (toneladas/semana) | costo (COP/semana)
#     1    | 1074                         | 3111202.75
#     2    | 2418                         | 4804980.75
# por municipio se puede construir más de un tipo de almacén, se busca minimizar
# el costo de construcción y satisfacer la demanda de alimentos para la población,
# sabiendo que la demanda de alimentos es de 0.87617 kg por persona.

# Los datos de arrendamiento de almacenes ya satisfacen la demanda de alimentos
# para la población del municipio, por lo que no se necesita construir almacenes,
# además se asume un factor de seguridad de 1.5 para la capacidad de los almacenes
# ya que se asume que la demanda de alimentos no es constante y puede variar.

# el modelo de optimización que se va a utilizar es el siguiente:
#   SETS:
#       i = {1, 2, ..., n} conjunto de municipios
#       j = {1, 2} conjunto de tipos de almacenes
#   PARAMETERS:
#       d_i = demanda de alimentos en el municipio i
#           d_i = poblacion_i * comida_per_capita
#       c_j = capacidad de almacenamiento del tipo j
#       costo_j = costo de construcción del tipo j
#   VARIABLES:
#       x_ij = cantidad de almacenes del tipo j en el municipio i
#   OBJECTIVE FUNCTION:
#       min sum(i, j) costo_j * x_ij
#   CONSTRAINTS:
#       sum(j) c_j * x_ij >= d_i | i = {1, 2, ..., n}
#       x_ij >= 0 | i = {1, 2, ..., n}, j = {1, 2}
#       x_ij = integer | i = {1, 2, ..., n}, j = {1, 2}


# proceso general:
#   1. leer los datos de los almacenes
#   2. leer los datos del pronóstico poblacional de los municipios para 2034
#   3. calcular la demanda de alimentos para cada municipio
#   5. resolver el problema de optimización
#   6. guardar los resultados en un archivo csv en
#       resultados/tablas/capacidad_y_costo.csv
def tipos_de_almacenes():
    data = {
        1: {"capacidad": 1074, "costo": 3111202.75},
        2: {"capacidad": 2418, "costo": 4804980.75},
    }
    return pd.DataFrame(data).T


def demanda_de_alimentos(poblacion: pd.Series, comida_per_capita: float) -> pd.Series:
    return pd.to_numeric(poblacion) * comida_per_capita * 7 * 1.5


def get_demanda_completo_e_imperfecto(comida_per_capita: float):
    poblacion_completo = pd.read_csv(
        "resultados/tablas/pronostico_poblacional/datos_completos.csv", index_col=0
    )
    poblacion_imperfecto = pd.read_csv(
        "resultados/tablas/pronostico_poblacional/datos_imperfectos.csv", index_col=0
    )

    demanda_completa = demanda_de_alimentos(
        poblacion_completo["Poblacion_2034"], comida_per_capita
    )
    demanda_imperfecta = demanda_de_alimentos(
        poblacion_imperfecto["Poblacion_2034"], comida_per_capita
    )

    data = {
        "demanda_completa": demanda_completa,
        "demanda_imperfecta": demanda_imperfecta,
    }
    return data


demandas = get_demanda_completo_e_imperfecto(comida_per_capita)
naves = tipos_de_almacenes()


def pl_capacidad_y_costos(
    d_i: pd.Series, c_j: pd.Series, costo_j: pd.Series, id_data: str
):
    """
    Soluciona el problema de optimización de capacidad y costos de almacenamiento
    de alimentos en cada municipio.

    Parameters
    ----------
    d_i : pd.Series
        demanda semanal de alimentos en cada municipio
    c_j : pd.Series
        capacidad de almacenamiento de cada tipo de almacén en toneladas
    costo_j : pd.Series
        costo de construcción de cada tipo de almacén en COP/semana

    Returns
    -------
    solucion : pd.DataFrame
        cantidad de almacenes de cada tipo en cada municipio
            solucion.columns = [(x_ij),capacidad, costo] en donde x_ij es un
                                        tuple con la cantidad de almacenes
                                        de cada tipo
            solucion.index = d_i.index

    Problema de optimización
    ------------------------

    SETS:
        i = {1, 2, ..., n} conjunto de municipios
        j = {1, 2} conjunto de tipos de almacenes
    PARAMETERS:
        d_i = demanda de alimentos en el municipio i
        c_j = capacidad de almacenamiento del tipo j
        costo_j = costo de construcción del tipo j
    VARIABLES:
        x_ij = cantidad de almacenes del tipo j en el municipio i
    OBJECTIVE FUNCTION:
        min sum(i, j) costo_j * x_ij
    CONSTRAINTS:
        sum(j) c_j * x_ij >= d_i | i = {1, 2, ..., n}
        x_ij >= 0 | i = {1, 2, ..., n}, j = {1, 2}
        x_ij = integer | i = {1, 2, ..., n}, j = {1, 2}
    """
    warnings.filterwarnings("ignore", category=UserWarning)
    I = d_i.index
    J = c_j.index

    # Crear el problema de optimización
    prob = pl.LpProblem("capacidad_y_costo", pl.LpMinimize)

    # Crear las variables
    x = pl.LpVariable.dicts("x", (I, J), lowBound=0, cat="Integer")

    # Crear la función objetivo
    prob += pl.lpSum([costo_j[j] * x[i][j] for i in I for j in J])

    # Crear las restricciones
    for i in I:
        prob += pl.lpSum([c_j[j] * x[i][j] for j in J]) >= d_i[i]

    # Resolver el problema
    prob.solve(
        solver=pl.PULP_CBC_CMD(
            logPath=f"resultados/logs/2. capacidad_y_costo/{id_data}.log",
        )
    )

    # Crear el dataframe de la solución
    solucion = pd.DataFrame(index=I, columns=["capacidad", "costo", "x_ij"])
    for i in I:
        solucion.loc[i, "capacidad"] = sum([c_j[j] * x[i][j].varValue for j in J])
        solucion.loc[i, "costo"] = sum([costo_j[j] * x[i][j].varValue for j in J])
        solucion.loc[i, "x_ij"] = [x[i][j].varValue for j in J]

    return solucion


for id_data, data in demandas.items():
    solucion = pl_capacidad_y_costos(data, naves["capacidad"], naves["costo"], id_data)
    solucion.to_csv(f"resultados/tablas/capacidad_y_costo/{id_data}.csv")
